In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/stream/

/content/drive/MyDrive/stream


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.7 MB/s eta 0:00:00


In [4]:
import pandas as pd

df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
def generate_text_and_labels(input_csv="IMDB Dataset.csv"):
    df = pd.read_csv(input_csv)
    
    df['label'] = [1 if x=="positive" else 0 for x in df['sentiment'] ]
    return df['review'].values, df['label'].values

In [6]:
texts, labels = generate_text_and_labels()

In [7]:
train_length = 40000
train_texts, train_labels = texts[:train_length], labels[:train_length]
test_texts, test_labels = texts[train_length:], labels[train_length:]

In [8]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [9]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [10]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

In [11]:
import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [12]:
train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

In [13]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [15]:
model.train()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [16]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

In [17]:
optim = AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
from tqdm import tqdm
for epoch in range(3):
    for batch in tqdm(train_dataloader):
        optim.zero_grad()
        input_ids= batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
    print(f"Loss for epoch {epoch} is {loss}")

model.eval()

100%|██████████| 2000/2000 [24:20<00:00,  1.37it/s]


Loss for epoch 0 is 0.26909852027893066


100%|██████████| 2000/2000 [24:21<00:00,  1.37it/s]


Loss for epoch 1 is 0.08318140357732773


100%|██████████| 2000/2000 [24:21<00:00,  1.37it/s]

Loss for epoch 2 is 0.2244129776954651


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [19]:
save_directory = "./"

In [20]:
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [21]:
from transformers import pipeline
classifier =pipeline('sentiment-analysis',model=model, tokenizer=tokenizer)

In [22]:
sample_test= test_dataset[0]
sample_test

{'input_ids': tensor([  101,  2034,  2125,  1045,  2215,  2000,  2360,  2008,  1045,  8155,
          4314,  2006,  1996,  2576,  4094,  1998,  1045,  2179,  1996,  3185,
          5805,  1012,  1045,  3266,  2000,  3422,  1996,  2878, 28844,  5643,
         29591,  1997,  1037,  2143,  1012,  2023,  3185,  7545,  1037,  2659,
          2000,  2434,  4784,  1012,  2748,  2009,  2001,  2434,  2947,  2026,
          1016,  3340,  2612,  1997,  1015,  1012,  2024,  2256,  2143,  4898,
          2008,  4895, 16748,  8082,  2008,  2027,  2064,  2069,  2272,  2039,
          2007,  2023,  1029,  1029,  3772,  2001,  9202,  1010,  1998,  1996,
          3494,  2020,  4406,  3085,  2005,  1996,  2087,  2112,  1012,  1996,
          2599,  3203,  1999,  1996,  2466,  2018,  2053,  2204, 11647,  2012,
          2035,  1012,  2027,  2081,  2014, 28939,  2046,  2070,  4066,  1997,
          1037,  2919,  3124,  1998,  1045,  2106,  2025,  2156,  2008,  2012,
          2035,  1012,  2672,  1045,  4

In [26]:
test_texts[1001], test_labels[1001]

('Acting was weak, but in a horror flick, I can live with that if the story is good. It wasn\'t. The initial event was an clumsy and obvious ploy to exploit most people\'s adoration of kids. OK, fine. Fast forward to the "place in the country" where they will recover emotionally. I like the revelation of the ghosts. OK, cool--this will be a supernatural kinda horror story, with rotting things partly in our world partly in...where ever. Then the action starts pulling like a three headed dog in a flurry of cats and birds--Is there an evil force trying to attack them directly? Is there an evil force trying to attack them INdirectly--make people do awful things they wouldn\'t really do? Oh, wait, no, maybe the whole REGION is some kind of psychic echo chamber where ambient discord can reverberate into murder? OK, hold on--maybe it\'s really just one little mentally tangled "Delbert"-style redneck boy who misses his Mommy and is on some kind of spree like a K-Tel Norman Bates knock off? Oh,